# Defining the layer stack

As mentioned previously, a layer stack defines the layers available to use as a designer. These layers correspond to process steps, whether that be etching of existing material, deposition of new layers, or doping. Materials may include layers for the substrate, the waveguides, cladding, metallization layers, etc., while process steps may include p- or n- doping regions, etch steps, and so on. 

The layer stack is usually defined in a GDS file. GDS files get translated into masks that are used to fabricate the chip.

A GDS file contains "layers" that are defined by a tuple of two numbers: the layer number and the datatype. Formally, the GDS specification states that the layer number and datatype can each be an integer in the range from 0 to 255, although some modern software may support higher values. 

While there is no formal or universal system, it's common for the layer number to define a material. It's almost universal to see one layer number dedicated to waveguides, another to a specific metal layer, and so on. 

Some fabs will use the datatype to define the process step. For example, the layer number 1 may be used to define the waveguide layer, while the datatype 1 may be used to define the etch step for the waveguide. In other cases, the datatype may be used to group together related objects; for example, all waveguides have the same layer number, but if you have multiple waveguide layers (e.g. silicon, silicon nitride, lithium niobate, etc.), each layer would get its own datatype.

Whichever system you decide to use is up to you (or your fab). The important thing is to be consistent.

## Silicon-on-insulator

<div style="text-align: center">
<img src="https://github.com/BYUCamachoLab/Photonics-Bootcamp/blob/main/book/images/soi_wg_xsection.png?raw=true" alt="Silicon-on-insulator waveguide cross-section" width="240">
</div>

The most common material system for integrated photonics is silicon-on-insulator (SOI). In this system, a thin layer of silicon is deposited on top of a layer of silicon dioxide (SiO2). The silicon dioxide layer is usually 2 microns thick, while the silicon layer is usually 220 nm thick. The silicon dioxide layer is called the "buried oxide" (BOX) layer, while the silicon layer is called the "device" layer.

Because the main principle of operation of a photonic waveguide is total internal reflection, the refractive index of the waveguide core must be higher than the refractive index of the cladding. In SOI, the silicon layer is the waveguide core, while the silicon dioxide layer is the cladding. The refractive index of silicon is approximately 3.48 (and is wavelength dependent), while the refractive index of silicon dioxide is usually taken to be 1.45, depending on the model.

```{tip}

An excellent source for refractive index data is [refractiveindex.info](https://refractiveindex.info/). For example, here's the pages for silicon and silicon dioxide:

* [silicon](https://refractiveindex.info/?shelf=main&book=Si&page=Pierce)
* [silicon dioxide](https://refractiveindex.info/?shelf=main&book=SiO2&page=Malitson)
```

## Defining a layer stack in gdsfactory

There are three tools to help you define a layer stack in gdsfactory:

* [``LayerMap``](layer-map): pairs layer names to GDS layer numbers and datatypes 
* [``LayerViews``](layer-views): defines visualization of design, like layer colors and fill patterns
* [``LayerStack``](layer-stack): information on thickness and position of each layer, for simulations or 3D models and cross-sections

(layer-map)=
## LayerMap

Layers define process steps or materials. A layer map is a dictionary that maps human-readable layer names to GDS layer numbers and datatypes. 

:::{note}

You can define a layer mapping from Python code or by reading an existing KLayout properties file (.lyp), if you're migrating an existing PDK to gdsfactory.
:::

There are a few utility layers that gdsfactory likes to have present if you're using any of the generic components or cross sections, as they automatically add geometries to these layers:

* ``DEVREC``: device recognition layer for connectivity checks
* ``PORT``: optical ports
* ``PORTE``: electrical ports
* ``SHOW_PORTS``: port markers
* ``LABEL``: add labels to grating couplers for automatic testing
* ``LABEL_INSTANCE``: for instance labels when reading from netlists
* ``TE``: TE polarization fiber marker
* ``TM``: TM polarization fiber marker

gdsfactory uses [Pydantic](https://docs.pydantic.dev/latest/) to define these configuration classes.

Below is a simple example layer map. It has a waveguide layer (including a partial etch layer), the utility layers, and a metal layer. Layers that you could add to your own PDK might include germanium layers (for photodiodes), silicon nitride waveguide layers, doping layers of varying concentrations (for pn junctions), more metal layers, capacitor layers, and vias of varying depths to access all of these. You may add other utility layers too, including {term}`DRC` exclude regions, dicing lanes, or layers reserved for showing errors and DRC violations. We won't do any of that here; it is left as an exercise to the reader.

*(This script is complete, it should run "as is")*

In [1]:
from pydantic import BaseModel
from gdsfactory.typings import Layer


class LayerMap(BaseModel):
    # waveguide layers
    WG: Layer = (1, 0)
    SLAB150: Layer = (2, 0)
    
    # utility layers
    DEVREC: Layer = (68, 0)
    PORT: Layer = (1, 10)
    PORTE: Layer = (1, 11)
    SHOW_PORTS: Layer = (1, 12)
    
    # label layers
    LABEL: Layer = (201, 0)
    LABEL_INSTANCES: Layer = (206, 0)
    TE: Layer = (203, 0)
    TM: Layer = (204, 0)
    TEXT: Layer = (66, 0)
    
    # electrical layers
    M1: Layer = (41, 0)
    VIA: Layer = (43, 0)
    
    class Config:
        frozen = True
        extra = "forbid"


LAYER = LayerMap()

2023-06-19 16:18:25.227 | INFO     | gdsfactory.config:__init__:204 - LogLevel: INFO


(layer-views)=
## LayerViews

Layer views define how your design will be visualized. This includes the color of each layer, the fill pattern, and whether the layer is visible or not. These settings are applied to the matplotlib plots, 3D views, cross-sections, and the settings can be exported to file for use in KLayout, too.

```yaml
LayerViews:
  Waveguide:
    layer: [1, 0]
    hatch_pattern: dotted
    width: 1
    color: "#ff9d9d"
  SLAB150:
    layer: [2, 0]
    layer_in_name: true
    hatch_pattern: coarsely dotted
    transparent: true
    width: 1
    color: "cyan"
  M1:
    layer: [41, 0]
    layer_in_name: true
    hatch_pattern: dotted
    width: 1
    color: "#01ff6b"
    brightness: -16
  VIA:
    layer: [40, 0]
    layer_in_name: true
    hatch_pattern: hollow
    width: 1
    color: "#cc4c00"
  LABEL_OPTICAL_IO:
    layer: [201, 0]
    layer_in_name: true
    hatch_pattern: hollow
    width: 1
    color: "blue"
  LABEL_SETTINGS:
    layer: [202, 0]
    layer_in_name: true
    hatch_pattern: hollow
    visible: false
    width: 1
    color: "magenta"
  TE:
    layer: [203, 0]
    layer_in_name: true
    transparent: true
    width: 1
    color: "blue"
  TM:
    layer: [204, 0]
    layer_in_name: true
    width: 1
    color: "red"
  LABEL_INSTANCES:
    layer: [206, 0]
    layer_in_name: true
    hatch_pattern: lightly left-hatched
    color: "blue"
  DevRec:
    layer: [68, 0]
    hatch_pattern: hollow
    visible: false
    transparent: true
    width: 1
    color: "#004080"
  PinRec:
    layer: [1, 10]
    hatch_pattern: hollow
    color: "#404040"
  Text:
    layer: [66, 0]
    hatch_pattern: hollow
    width: 1
    color: "blue"
  XSECTION:
    group_members:
      XS_BOX:
        layer: [300, 0]
        layer_in_name: true
        width: 1
        color: "#f3ff80"
        hatch_pattern: solid
      XS_SI:
        layer: [301, 0]
        layer_in_name: true
        width: 1
        color: "black"
        hatch_pattern: solid
      XS_SI_SLAB:
        layer: [313, 0]
        layer_in_name: true
        width: 1
        color: "#80a8ff"
        hatch_pattern: solid
      XS_OVERLAY:
        layer: [311, 0]
        layer_in_name: true
        width: 1
        color: "blue"
        hatch_pattern: solid
      XS_OX_SI:
        layer: [302, 0]
        layer_in_name: true
        width: 1
        color: "#f3ff80"
        hatch_pattern: solid
      XS_VIA:
        layer: [303, 0]
        layer_in_name: true
        width: 1
        color: "grey"
        hatch_pattern: solid
      XS_M1:
        layer: [304, 0]
        layer_in_name: true
        width: 1
        color: "green"
        hatch_pattern: solid
      XS_OXIDE_M1:
        layer: [305, 0]
        layer_in_name: true
        width: 1
        color: "#f3ff80"
        hatch_pattern: solid
      XS_VIA:
        layer: [308, 0]
        layer_in_name: true
        width: 1
        color: "grey"
        hatch_pattern: solid
  SHOW_PORTS:
    layer: [1, 12]
    layer_in_name: true
    hatch_pattern: lightly left-hatched
    color: "#ff80a8"
  WG_PIN:
    layer: [1, 10]
    layer_in_name: true
    hatch_pattern: lightly left-hatched
    color: "#ff80a8"
```

In [3]:
from gdsfactory.config import PATH

In [9]:
PATH.klayout_lyp

PosixPath('/home/sequoia/git/Photonics-Bootcamp/env/lib/python3.11/site-packages/gdsfactory/generic_tech/klayout/tech/generic_tech.lyp')

(layer-stack)=
## LayerStack

## Exercises

1. Create a layer stack that includes a silicon waveguide layer, a silicon nitride waveguide layer, doping layers of three concentrations for each of n-type and p-type (n, np, npp, p, pp, ppp), and two metal layers. Add a germanium layer directly above the silicon waveguides for photodiodes. Be sure to include the [utility layers](https://gdsfactory.github.io/gdsfactory/notebooks/08_pdk.html#layers) required by gdsfactory in your stack. Select your own layer numbers.